<a href="https://colab.research.google.com/github/jog822/CRISPR-GEM/blob/main/CRISPR_GEM_Github.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Model

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
import pandas as pd
import numpy as np
import os
import gzip
import warnings
import sklearn
from scipy import stats
from tables import index
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.preprocessing import StandardScaler
from matplotlib import pyplot as plt

import tensorflow as tf
keras = tf.keras
from tensorflow.keras import Model
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from tensorflow.keras.metrics import sparse_categorical_accuracy
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import l2
from tensorflow.keras.layers import Input, Dense, Lambda
from tensorflow.keras.metrics import MeanSquaredError

from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping
from keras.metrics import MeanSquaredError

Mounted at /content/drive


In [ ]:
folder_path = '/content/drive/MyDrive/GEM'
if not os.path.exists(folder_path):
    os.makedirs(folder_path)
    print(f"Folder created at {folder_path}")
else:
    print(f"Folder already exists at {folder_path}")

def extract_transcript_gene_mapping(gtf_file):
    transcript_to_gene = {}
    if gtf_file.endswith('.gz'):
        import gzip
        open_func = gzip.open
    else:
        open_func = open

    with open_func(gtf_file, 'rt', encoding='utf-8') as f:
        for line in f:
            if line.startswith('#'):
                continue
            fields = line.strip().split('\t')
            if fields[2] == 'transcript':
                attributes = dict(item.strip().split(' ') for item in fields[8].split(';') if item.strip())
                transcript_id = attributes['transcript_id'].strip('"')
                gene_symbol = attributes['gene_name'].strip('"')
                transcript_to_gene[transcript_id] = gene_symbol
    return transcript_to_gene

def get_counts(df, prefix, transcript_to_gene):
    df4=df.copy()
    df4.index=df4.index.map(transcript_to_gene)
    df5 = df4[~df4.index.str.contains('ENSG', na=False)]
    df2=df5.groupby(df5.index, axis=0).max()
    df3=df2.T
    input_file="/content/drive/MyDrive/GEM/"+prefix+"_input_list.csv"
    inputs=pd.read_csv(input_file)
    i=inputs['inputs'].tolist()
    i2 = [value for value in i if value in df3.columns]
    input_df=df3[i2]
    input_df.fillna(0, inplace=True)
    output_file="/content/drive/MyDrive/GEM/"+prefix+"_output_list.csv"
    outputs=pd.read_csv(output_file)
    o=outputs['outputs'].tolist()
    o2 = [value for value in o if value in df3.columns]
    o3=list(set(o2) - set(i2))
    print('Input Shape: '+str(len(i2)))
    print('Output Shape: '+str(len(o2)))
    print('New Output Shape: '+str(len(o3)))
    output_df=df3[o3]
    output_df.fillna(0, inplace=True)
    return (input_df, output_df)

def traiNN(input_df, output_df, prefix):
  output=np.log1p(output_df+1)
  inputs=np.log1p(input_df+1)

  test_size = 0.15
  X_train1, X_test1, Y_train1, Y_test1 = train_test_split(inputs,output, test_size=test_size, random_state=23)
  X_train = tf.constant(X_train1.values, dtype=tf.float32)
  Y_train = tf.constant(Y_train1.values, dtype=tf.float32)
  X_test = tf.constant(X_test1.values, dtype=tf.float32)
  Y_test = tf.constant(Y_test1.values, dtype=tf.float32)

  %matplotlib inline
  %config InlineBackend.figure_format = 'retina'
  #@keras.saving.register_keras_serializable(name="r_squared")
  def r_squared(y_true, y_pred):
      SS_res = tf.reduce_sum(tf.square(y_true - y_pred))
      SS_tot = tf.reduce_sum(tf.square(y_true - tf.reduce_mean(y_true)))
      return 1 - (SS_res / (SS_tot + tf.keras.backend.epsilon()))
  maxy=max(len(input_df.columns),len(output_df.columns))
  model = Sequential()
  model.add(Dense(10000, activation='relu', input_shape=(len(input_df.columns),)))
  model.add(Dense(10000, activation='relu'))
  model.add(Dense(10000, activation='relu'))
  model.add(Dense(len(output_df.columns)))
  callback = keras.callbacks.EarlyStopping(monitor='val_loss', patience=10)
  model.compile(Adam(learning_rate=0.0001, beta_1=0.9, beta_2=0.999, epsilon=1e-8), loss='mean_squared_error', metrics=[r_squared])
  epochs = 200
  batch_size = 50
  validation_split = 0.15
  history = model.fit(X_train, Y_train, batch_size=batch_size, epochs=epochs,
                      callbacks=[callback],validation_split=validation_split,
                      verbose=0)
  predictions = model.predict(X_train)
  history_dict = history.history
  loss_values = history_dict['loss']
  val_loss_values = history_dict['val_loss']
  r_squared_values = history_dict['r_squared']
  val_r_squared_values = history_dict['val_r_squared']
  plt.figure(figsize=(10, 5))
  plt.subplot(1, 2, 1)
  plt.plot(loss_values, 'bo', label='training loss')
  plt.plot(val_loss_values, 'r', label='validation loss')
  plt.xlabel('Epochs')
  plt.ylabel('Loss (MSE)')
  plt.ylim(top=1, bottom=0)
  plt.legend()
  r2 = "r²"
  plt.subplot(1, 2, 2)
  plt.plot(r_squared_values, 'bo', label='training '+r2)
  plt.plot(val_r_squared_values, 'r', label='validation '+r2)
  plt.xlabel('Epochs')
  plt.ylabel(r2)
  plt.ylim(0, 1)
  plt.legend()
  plt.savefig('/content/drive/MyDrive/GEM/'+ prefix + "/"+prefix+'.png')
  plt.show()
  predictions = model.predict(X_test)
  r2 = r_squared(Y_test, predictions)
  print(f"R-squared (R2) Score on Testing Data: {r2.numpy()}")
  mse = MeanSquaredError()(Y_test, predictions)
  print(f"Mean Squared Error (MSE) on Testing Data: {mse.numpy()}")
  model.save('/content/drive/MyDrive/GEM/'+ prefix + "/"+prefix+'.keras')
  plt.close()
  y_test_flattened = Y_test.numpy().flatten()
  y_pred2 = model.predict(X_test)
  y_pred_flattened = y_pred2.flatten()
  plt.scatter(y_test_flattened, y_pred_flattened)
  plt.xlabel('Actual')
  plt.ylabel('Predicted')
  plt.title('Actual vs Predicted Values')
  plt.show()
  plt.savefig('/content/drive/MyDrive/GEM/'+ prefix +"/"+prefix+ '_dotplot.png')
  #Calculate a P Value
  n = len(y_test_flattened)
  k = X_test.shape[1]
  ss_tot = np.sum((y_test_flattened - np.mean(y_test_flattened)) ** 2)
  ss_res = np.sum((y_test_flattened - y_pred_flattened) ** 2)
  ss_reg = ss_tot - ss_res
  ms_reg = ss_reg / k
  ms_res = ss_res / (n - k - 1)
  f_statistic = ms_reg / ms_res
  p_value = 1 - stats.f.cdf(f_statistic, k, n - k - 1)
  print(f"P-value: {p_value}")
  return model, input_df, output_df, r2, mse, p_value

def evaluate_her(model, input_df, output_df, input_indices, output_indices):
  input_df=input_df.fillna(0, inplace=False)
  output_df=output_df.fillna(0, inplace=False)
  input_df=pd.DataFrame(np.log1p(input_df+1), columns=input_df.columns, index=input_df.index)
  output_df=pd.DataFrame(np.log1p(output_df+1), columns=output_df.columns, index=output_df.index)
  evaluater_input=input_df[input_df.index.isin(input_indices)]
  evaluater_output=output_df[output_df.index.isin(output_indices)]
  others=output_df[output_df.index.isin(input_indices)]
  bigE2=input_df[input_df.index.isin(output_indices)]
  return evaluater_input, evaluater_output, others, bigE2

def process_and_evaluate(input_data, Evaluater, other, bigE, model, prefix, prefix1, output_df, CRISPR):
    k=30
    results=[]
    from sklearn import preprocessing
    min_max_scaler=preprocessing.MinMaxScaler()
    foldi=None
    folda=None
    foldko=None
    if CRISPR=='CRISPRko':
      resultko=[]
      foldko=0
      for col in input_data.columns:
            i4 = input_data.copy()
            i4[col] = input_data[col] * foldko
            perterb = i4.copy()
            E=bigE.copy()
            for row in perterb.index:
                here = pd.DataFrame(perterb.loc[row]).T
                trunk = pd.DataFrame(input_data.loc[row]).T
                chang = model.predict(trunk, verbose=0).T
                other2 = pd.DataFrame(other.loc[row]).T
                predictions = model.predict(here, verbose=0)
                predictions1 = predictions.T
                other3 = other2.T
                for row2 in Evaluater.index:
                  E1=pd.DataFrame(E.loc[row2]).T
                  E2=model.predict(E1, verbose=0) #####
                  r2 = r2_score(E2.T, predictions1)
                  mse = mean_squared_error(E2.T, predictions1)
                  num2=abs(predictions1 - E2.T)/(other3.values+0.0001)
                  num2=abs(predictions1 - E2.T)/(chang+0.0001)
                  score = sum(num2) / len(output_df.columns)
                  resultko.append({'Gene': col, 'n': row, 'output': row2, 'r2': float(r2), 'MSE': float(mse), 'Score': float(score)})
      resultko_df=pd.DataFrame(resultko)
      view3=resultko_df.copy()
      view4=view3.drop(columns=['n', 'output'])
      view2=view4.groupby('Gene').mean()
      view2_r2=view2.sort_values(by='r2', ascending=False)
      view2_MSE=view2.sort_values(by='MSE', ascending=True)
      view2_score=view2.sort_values(by='Score', ascending=True)
      view2_score.to_csv('/content/drive/MyDrive/GEM/'+prefix+"/"+prefix1+'_output.csv')
    elif CRISPR=='CRISPRa':
      folda=10
      k=15
      for col in input_data.columns:
        i2 = input_data.copy()
        i2[col] = input_data[col] * (1 + k / input_data[col])
        perterb = i2.copy()
        E=bigE.copy()
        for row in perterb.index:
            here = pd.DataFrame(perterb.loc[row]).T
            other2 = pd.DataFrame(other.loc[row]).T
            trunk = pd.DataFrame(input_data.loc[row]).T
            chang = model.predict(trunk, verbose=0).T
            predictions = model.predict(here, verbose=0)
            predictions1 = predictions.T
            other3 = other2.T
            for row2 in Evaluater.index:
              eval=pd.DataFrame(Evaluater.loc[row2]).T
              E1=pd.DataFrame(E.loc[row2]).T
              E2=model.predict(E1, verbose=0) #####
              # r2 = r2_score(eval.T, predictions1)
              r2 = r2_score(E2.T, predictions1)
              # mse = mean_squared_error(eval.T, predictions1)
              mse = mean_squared_error(E2.T, predictions1)
              # num2=abs(predictions1 - E2.T)/(other3.values+0.0001)
              num2=abs(predictions1 - E2.T)/(chang+0.0001)
              score = sum(num2) / len(output_df.columns)
              results.append({'Gene': col, 'n': row, 'output': row2, 'r2': float(r2), 'MSE': float(mse), 'Score': float(score)})
      results_df = pd.DataFrame(results)
      view3=results_df.copy()
      view4=view3.drop(columns=['n', 'output'])
      view2=view4.groupby('Gene').mean()
      view2_r2=view2.sort_values(by='r2', ascending=False)
      view2_MSE=view2.sort_values(by='MSE', ascending=True)
      view2_score=view2.sort_values(by='Score', ascending=True)
      view2_score.to_csv('/content/drive/MyDrive/GEM/'+prefix+"/"+prefix1+'_output.csv')
      view3.to_csv('/content/drive/MyDrive/GEM/'+prefix+"/"+prefix1+'_output_NEW.csv')
    elif CRISPR=='CRISPRi':
      resulti=[]
      foldi=0.1
      for col in input_data.columns:
            i4 = input_data.copy()
            i4[col] = input_data[col]/(1+np.log1p(input_data[col]))
            # perterb = pd.DataFrame(min_max_scaler.fit_transform(i4), columns=i4.columns, index=i4.index)
            perterb=i4.copy()
            E=bigE.copy()
            for row in perterb.index:
                here = pd.DataFrame(perterb.loc[row]).T
                other2 = pd.DataFrame(other.loc[row]).T
                trunk = pd.DataFrame(input_data.loc[row]).T
                chang = model.predict(trunk, verbose=0).T
                predictions = model.predict(here, verbose=0)
                predictions1 = predictions.T
                other3 = other2.T
                for row2 in Evaluater.index:
                  E1=pd.DataFrame(E.loc[row2]).T
                  E2=model.predict(E1, verbose=0) #####
                  r2 = r2_score(E2.T, predictions1)
                  mse = mean_squared_error(E2.T, predictions1)
                  num2=abs(predictions1 - E2.T)/(chang+0.0001)
                  score = sum(num2) / len(output_df.columns)
                  resulti.append({'Gene': col, 'n': row, 'output': row2, 'r2': float(r2), 'MSE': float(mse), 'Score': float(score)})
      resulti_df=pd.DataFrame(resulti)
      view3=resulti_df.copy()
      view4=view3.drop(columns=['n', 'output'])
      view2=view4.groupby('Gene').mean()
      view2_r2=view2.sort_values(by='r2', ascending=False)
      view2_MSE=view2.sort_values(by='MSE', ascending=True)
      view2_score=view2.sort_values(by='Score', ascending=True)
      view2_score.to_csv('/content/drive/MyDrive/GEM/'+prefix+"/"+prefix1+'_output.csv')
    return


def analyze_methods(prefix, CRISPR, input_indices, output_indices):
    result_df1=pd.read_csv('/content/drive/MyDrive/GEM/TPM.csv', index_col=0)
    gtf_file = "/content/drive/MyDrive/GEM/gencode.v45.chr_patch_hapl_scaff.annotation.gtf/gencode.v45.chr_patch_hapl_scaff.annotation.gtf"
    transcript_to_gene = extract_transcript_gene_mapping(gtf_file)
    threshold = result_df1.shape[1] / 3
    filtered_df = result_df1[result_df1.sum(axis=1) >= threshold]
    model_scores=[]
    os.makedirs('/content/drive/MyDrive/GEM/'+ prefix, exist_ok=True)
    suffixes = ["_1_", "_2_", "_3_", "_4_", "_5_", "_6_"]
    prefix_list = [prefix + suffix for suffix in suffixes]
    for prefix1 in prefix_list:
      input_df, output_df= get_counts(filtered_df, prefix, transcript_to_gene)
      model, input_df, output_df, r2, mse, p_value =traiNN(input_df, output_df, prefix)
      model_scores.append({'Prefix': prefix1, 'r2': float(r2), 'mse': float(mse), 'p_val': float(p_value)})
      Tester, evaluater_outputs, other, bigE= evaluate_her(model, input_df, output_df, input_indices, output_indices)
      final_save=prefix1+'_view2_score'
      final_res=prefix1+'_result_df'
      with warnings.catch_warnings():
          warnings.filterwarnings("ignore", category=DeprecationWarning)
          process_and_evaluate(Tester, evaluater_outputs, other, bigE, model,prefix, prefix1, output_df, CRISPR)
      df = pd.DataFrame(model_scores)
      df.to_csv('/content/drive/MyDrive/GEM/'+ prefix +"/"+prefix1+ '_train_test_scores.csv', index=False)
    return df

Folder created at /content/drive/MyDrive/GEM


# Run it...

In [ ]:
#Datasets used for publication:
## input_indices=['SRR22560355', 'SRR13018743', 'SRR21009234'] # MSC for chondrogenesis
## input_indices=['SRR2453345', 'SRR2453349', 'SRR2453348'] # iPSC for Treg differentiation
## input_indices=['SRR7107496', 'SRR7107494', 'SRR7107498'] # OA Cartilage

## output_indices=['SRR6337333', 'SRR6337334', 'SRR6337335'] # # Juvenile Cartilage for Chondrogenesis
## output_indices=['SRR7107486', 'SRR7107484', 'SRR7107481'] # Healthy Cart for OA compare
## output_indices=['SRR25253642', 'SRR25253654', 'SRR25253978'] #Tregs for differentiation

prefix='' # Same string you typed into GUI save box,
# Make sure input_list and output_list for that string are moved to GEM folder in google drive
CRISPR_type='' # CRISPRa, CRISPRi, CRISPRko, OR CRISPRki
analyze_methods(prefix, CRISPR_type, input_indices, output_indices)

# View the results:


In [ ]:
import os
import pandas as pd
from scipy.stats import gmean
parent_directory = '/content/drive/MyDrive/GEM'
dfs = []
suffixes = ["_1_", "_2_", "_3_", "_4_", "_5_", "_6_"]
prefix_list = [prefix + suffix for suffix in suffixes]
for prefix1 in prefix_list:
    csv_file = os.path.join(parent_directory, prefix, prefix1 + "_output.csv")
    if os.path.exists(csv_file):
        df = pd.read_csv(csv_file)
        # print(df)
        df.rename(columns={"Score": prefix1}, inplace=True)
        df['rank']=range(1, len(df) + 1)
        df.sort_values('Gene', ascending=True, inplace=True)
        df.set_index('Gene', inplace=True)
        df=df.drop(columns=['MSE', 'r2', 'rank'])
        dfs.append(df)
merged_df =pd.concat(dfs, axis=1)
merged_df['average'] = merged_df.apply(lambda row: gmean(row), axis=1)
merged_df.sort_values(by='average', ascending=True).head(50)

#OR Search for specific genes
# rows_containing_tgf = merged_df[merged_df.index.str.contains('COL2', case=False)]
# rows_containing_tgf
